In [1]:
import pandas as pd
import numpy as np

from math import radians, cos, sin, asin, sqrt
def haversine(df):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    miles_list = []
    for i in df.index.tolist():
        df.loc[i,'start station longitude'], df.loc[i,'start station latitude'], \
               df.loc[i,'end station longitude'], df.loc[i,'end station latitude'] = map(radians, 
        [df.loc[i,'start station longitude'], df.loc[i,'start station latitude'], df.loc[i,'end station longitude'], 
         df.loc[i,'end station latitude']])
        # haversine formula 
        dlon = df.loc[i,'end station longitude'] - df.loc[i,'start station longitude'] 
        dlat = df.loc[i,'end station latitude'] - df.loc[i,'start station latitude'] 
        a = sin(dlat/2)**2 + cos(df.loc[i,'start station latitude']) * cos(df.loc[i,'end station latitude']) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        # Radius of earth in miles is 3958.8
        miles = 3958.8 * c
        miles_list.append(miles)
    return pd.Series(miles_list)

In [2]:
rider_df = pd.read_csv(r'C:\Users\mmotd\OneDrive\Documents\Boot Camp Files\Capstone\downsampled_rider.csv.gz')

In [3]:
rider_df = rider_df.drop('Unnamed: 0', axis = 1)

In [4]:
rider_df['weekend'] = pd.Series([1 if (i == 'Saturday') | (i == 'Sunday') else 0 for i in rider_df.dayofweek])

In [5]:
rider_df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station latitude     float64
start station longitude    float64
end station latitude       float64
end station longitude      float64
year                         int64
start_date                  object
stop_date                   object
start_hour                   int64
stop_hour                    int64
start_min                    int64
stop_min                     int64
season                      object
dayofweek                   object
start_interval              object
stop_interval               object
Sample_num                   int64
weekend                      int64
dtype: object

In [6]:
rider_df['miles'] = haversine(rider_df.loc[:,['start station longitude', 'start station latitude', 
                              'end station longitude', 'end station latitude']])

In [7]:
rider_df

,tripduration,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,year,start_date,stop_date,...,stop_hour,start_min,stop_min,season,dayofweek,start_interval,stop_interval,Sample_num,weekend,miles
0,7728,2017-09-01 00:04:08,2017-09-01 02:12:56,40.708621,-74.007222,40.708621,-74.007222,2017,2017-09-01,2017-09-01,...,2,0,0,fall,Friday,0:00,2:00,0,0,0.000000
1,530,2017-09-01 00:06:19,2017-09-01 00:15:10,40.760301,-73.998842,40.751873,-73.977706,2017,2017-09-01,2017-09-01,...,0,0,0,fall,Friday,0:00,0:00,0,0,1.250144
2,497,2017-09-01 00:21:00,2017-09-01 00:29:18,40.684751,-73.999173,40.697601,-73.993446,2017,2017-09-01,2017-09-01,...,0,20,20,fall,Friday,0:20,0:20,0,0,0.937175
3,156,2017-09-01 00:28:28,2017-09-01 00:31:04,40.725029,-73.990697,40.724677,-73.987834,2017,2017-09-01,2017-09-01,...,0,20,20,fall,Friday,0:20,0:20,0,0,0.151841
4,141,2017-09-01 00:40:33,2017-09-01 00:42:55,40.793770,-73.971888,40.791956,-73.968087,2017,2017-09-01,2017-09-01,...,0,40,40,fall,Friday,0:40,0:40,0,0,0.235062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103300,776,2020-01-31 23:35:04.539,2020-01-31 23:48:01.244,40.742388,-73.997262,40.760683,-73.984527,2020,2020-01-31,2020-01-31,...,23,20,20,winter,Friday,23:20,23:20,0,0,1.429086
1103301,778,2020-01-31 23:35:10.096,2020-01-31 23:48:08.387,40.759291,-73.988597,40.740343,-73.989551,2020,2020-01-31,2020-01-31,...,23,20,20,winter,Friday,23:20,23:20,0,0,1.310151
1103302,116,2020-01-31 23:37:55.710,2020-01-31 23:39:51.979,40.802557,-73.949078,40.804038,-73.945925,2020,2020-01-31,2020-01-31,...,23,20,20,winter,Friday,23:20,23:20,0,0,0.194098
1103303,149,2020-01-31 23:56:28.590,2020-01-31 23:58:58.200,40.779668,-73.980930,40.777057,-73.978985,2020,2020-01-31,2020-01-31,...,23,40,40,winter,Friday,23:40,23:40,0,0,0.207121
